In [3]:
import collections
import traceback
from typing import Union, Tuple, List
from dataclasses import replace
import numpy as np
from pvtrace.geometry.utils import (
    distance_between, close_to_zero, points_equal, flip, angle_between
)
from pvtrace.material.distribution import Distribution
import logging
logger = logging.getLogger(__name__)


gaussian = lambda x, c1, c2, c3: c1*np.exp(-((c2 - x)/c3)**2)

bandgap = lambda x, cutoff, alpha: (1 - np.heaviside(x-cutoff, 0.5)) * alpha

def isotropic():
    g1, g2 = np.random.uniform(0, 1, 2)
    phi = 2 * np.pi * g1
    mu = 2 * np.pi * g2 - 1 # mu = cos(theta)
    theta = np.arccos(mu)
    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(theta)
    return (x, y, z)


def henyey_greenstein(g=0.0):
    # https://www.astro.umd.edu/~jph/HG_note.pdf
    p = np.random.uniform(0, 1)
    s = 2 * p - 1
    mu = 1/(2*g) * (1 + g**2 - ((1 - g**2)/(1 + g*s))**2)
    # Inverse is not defined at g=0 but in the limit 
    # tends to the isotropic case.
    if close_to_zero(g):
        return isotropic()
    phi = 2 * np.pi * np.random.uniform()
    theta = np.arccos(mu)
    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = mu
    return (x, y, z)


def fresnel_reflectivity(angle, n1, n2):
    # Catch TIR case
    if n2 < n1 and angle > np.arcsin(n2/n1):
        return 1.0
    c = np.cos(angle)
    s = np.sin(angle)
    k = np.sqrt(1 - (n1/n2 * s)**2)
    Rs1 = n1 * c - n2 * k
    Rs2 = n1 * c + n2 * k
    Rs = (Rs1/Rs2)**2
    Rp1 = n1 * k - n2 * c
    Rp2 = n1 * k + n2 * c
    Rp = (Rp1/Rp2)**2
    r = 0.5 * (Rs + Rp)
    return r


def specular_reflection(direction, normal):
    print("Reflection", (direction, normal))
    
    vec = np.array(direction)
    normal = np.array(normal)
    if np.dot(normal, direction) < 0.0:
        normal = flip(normal)
    d = np.dot(normal, vec)
    reflected_direction = vec - 2 * d * normal
    return reflected_direction


def fresnel_refraction(direction, normal, n1, n2):
    print("Refraction", (direction, normal, n1, n2))
    vector = np.array(direction)
    normal = np.array(normal)
    if np.dot(normal, direction) < 0.0:
        normal = flip(normal)
    n = n1/n2
    dot = np.dot(vector, normal)
    c = np.sqrt(1 - n**2 * (1 - dot**2))
    sign = 1
    if dot < 0.0:
        sign = -1
    refracted_direction = n * vector + sign*(c - sign*n*dot) * normal
    return refracted_direction


class Surface(object):
    """ Surface of a geometry which handles details of reflection at an interface.
    """

    def is_reflected(self, ray, geometry, container, adjacent):
        """ Monte-Carlo sampling. Default is to transmit.
        """
        return False

    def reflect(self, ray, geometry, container, adjacent):
        """ Specular reflection.
        """
        normal = geometry.normal(ray.position)
        direction = ray.direction
        print("Incident ray", direction)
        reflected_direction = specular_reflection(direction, normal)
        print("Reflected ray", reflected_direction)
        ray = replace(
            ray,
            position=ray.position, 
            direction=tuple(reflected_direction.tolist())
        )
        return ray

    def transmit(self, ray, geometry, container, adjacent):
        """ Simply propgate."""
        return ray


class FresnelSurface(Surface):
    """ Implements reflection and refraction at an interface of two dielectrics.
    """
    
    def is_reflected(self, ray, geometry, container, adjacent):
        """ Monte-Carlo sampling. Default is to transmit.
        """
        # to-do: express ray in local coordinate system
        n1 = container.geometry.material.refractive_index
        n2 = adjacent.geometry.material.refractive_index
        # Be tolerance with definition of surface normal
        normal = geometry.normal(ray.position)
        if np.dot(normal, ray.direction) < 0.0:
            normal = flip(normal)
        angle = angle_between(normal, np.array(ray.direction))
        gamma = np.random.uniform()
        return gamma < fresnel_reflectivity(angle, n1, n2)

    def transmit(self, ray, geometry, container, adjacent):
        """ Refract through the interface.
        """
        # to-do: express ray in local coordinate system
        n1 = container.geometry.material.refractive_index
        n2 = adjacent.geometry.material.refractive_index
        # Be tolerance with definition of surface normal
        normal = geometry.normal(ray.position)
        if np.dot(normal, ray.direction) < 0.0:
            normal = flip(normal)
        refracted_direction = fresnel_refraction(ray.direction, normal, n1, n2)
        ray = replace(
            ray,
            position=ray.position, 
            direction=tuple(refracted_direction.tolist())
        )
        return ray


class Hit(object):
    """ Describes the hit location of a ray.
    """
    def __init__(self, node, position, distance):
        #: The node that has been hit
        self.node = node
        #: The hit point on node's surface
        self.position = position 
        #: Distance the ray tracelled to reach the point
        self.distance = distance


class Component(object):
    """ Base class for all things that can be added to a host material.
    """
    def __init__(self):
        super(Component, self).__init__()

    def is_radiative(self, ray):
        return False


class Scatterer(Component):
    """Describes a scatterer center with attenuation coefficient per unit length."""
    
    def __init__(self, coefficient, x=None, quantum_yield=1.0, phase_function=None):
        super(Scatterer, self).__init__()
        
        # Make absorption/scattering spectrum distribution
        self._coefficient = coefficient
        if coefficient is None:
            raise ValueError("Coefficient must be specified.")
        elif isinstance(coefficient, (float, np.float)):
            self._abs_dist = Distribution(x=None, y=coefficient)
        elif isinstance(coefficient, np.ndarray):
            self._abs_dist = Distribution(x=coefficient[:, 0], y=coefficient[:, 1])
        elif isinstance(coefficient, (list, tuple)):
            if x is None:
                raise ValueError("Requires `x`.")
            self._abs_dist = Distribution.from_functions(x, coefficient)
            
        self.quantum_yield = quantum_yield
        self.phase_function = phase_function if phase_function is not None else isotropic

    def coefficient(self, wavelength):
        value = self._abs_dist(wavelength)
        if value < 0:
            import pdb; pdb.set_trace()
        return value

    def is_radiative(self, ray):
        """ Monte-Carlo sampling to determine of the event is radiative.
        """
        return np.random.uniform() < self.quantum_yield
    
    def emit(self, ray: "Ray") -> "Ray":
        """ Change ray direction or wavelength based on physics of the interaction.
        """
        direction = self.phase_function()
        ray = replace(
            ray,
            direction=direction
        )
        return ray


class Absorber(Scatterer):
    """ Absorption only.
    """
    
    def __init__(self, coefficient, x=None):
        super(Absorber, self).__init__(coefficient, x=x, quantum_yield=0.0, phase_function=None)

    def is_radiative(self, ray):
        return False


class Luminophore(Scatterer):
    """Describes molecule, nanocrystal or material which absorbs and emits light."""
    
    def __init__(self, coefficient, emission=None, x=None, quantum_yield=1.0, phase_function=None):
        super(Luminophore, self).__init__(
            coefficient,
            x=x,
            quantum_yield=quantum_yield,
            phase_function=phase_function
        )
        
        # Make emission spectrum distribution
        self._emission = emission
        if emission is None:
            self._ems_dist = Distribution.from_functions(
                x, [lambda x: gaussian(x, 1.0, 600.0, 40.0)]
            )
        elif isinstance(emission, np.ndarray):
            self._ems_dist = Distribution(x=emission[:, 0], y=emission[:, 1])
        elif isinstance(emission, (tuple, list)):
            if x is None:
                raise ValueError("Requires `x`.")
            self._ems_dist = Distribution.from_functions(x, emission)
        else:
            raise ValueError("Lumophore `emission` arg has wrong type.")

    def emit(self, ray: "Ray") -> "Ray":
        """ Change ray direction or wavelength based on physics of the interaction.
        """
        theta = np.random.uniform(0, 2 * np.pi)
        phi = np.arccos(2 * np.random.uniform(0.0, 1.0) - 1)
        x = np.sin(phi) * np.cos(theta)
        y = np.sin(phi) * np.sin(theta)
        z = np.cos(phi)
        direction = (x, y, z)
        dist = self._ems_dist
        p1 = dist.lookup(ray.wavelength)
        p2 = 1.0
        max_wavelength = dist.sample(1.0)
        gamma = np.random.uniform(p1, p2)
        wavelength = dist.sample(gamma)
        ray = replace(
            ray,
            direction=direction,
            wavelength=wavelength
        )
        return ray
    

class Material(object):
    
    def __init__(self, refractive_index: float, components=None):
        self.refractive_index = refractive_index
        self.components = [] if components is None else components

    # Cache this function!
    def total_attenutation_coefficient(self, wavelength: float) -> float:
        coefs = [x.coefficient(wavelength) for x in self.components]
        print(coefs)
        alpha = np.sum(coefs)
        return alpha

    def is_absorbed(self, ray, full_distance) -> Tuple[bool, float]:
        distance = self.penetration_depth(ray.wavelength)
        return (distance < full_distance, distance)

    def penetration_depth(self, wavelength: float) -> float:
        """ Monte-Carlo sampling to find penetration depth of ray due to total
            attenuation coefficient of the material.
        
            Arguments
            --------
            wavelength: float
                The ray wavelength in nanometers.

            Returns
            -------
            depth: float
                The penetration depth in centimetres or `float('inf')`.
        """
        alpha = self.total_attenutation_coefficient(wavelength)
        logger.info('Got alpha({}) = {}'.format(wavelength, alpha))
        if np.isclose(alpha, 0.0):
            return float('inf')
        elif not np.isfinite(alpha):
            return 0.0
        # Sample exponential distribution
        depth = -np.log(1 - np.random.uniform())/alpha
        return depth

    def component(self, wavelength: float) -> Union[Scatterer, Luminophore]:
        """ Monte-Carlo sampling to find which component captures the ray.
        """
        coefs = [x.coefficient(wavelength) for x in self.components]
        if np.any(coefs < 0.0):
            raise ValueError("Must be positive.")
        count = len(self.components)
        bins = list(range(0, count + 1))
        cdf = np.cumsum(coefs)
        pdf = cdf / max(cdf)
        pdf = np.hstack([0, pdf[:]])
        pdfinv_lookup = np.interp(np.random.uniform(), pdf, bins)
        index = int(np.floor(pdfinv_lookup))
        component = self.components[index]
        return component


def find_container(intersections):
    # Find container node that has two properties:
    # 1. only appears in the intersection list once
    # 2. of the above is the has closest intersection to ray position
    if len(intersections) == 1:
        return intersections[0].hit
    count = collections.Counter([x.hit for x in intersections]).most_common()
    candidates = [x[0] for x in count if x[1] == 1]
    pairs = []
    for intersection in intersections:
        node = intersection.hit
        if node in candidates:
            pairs.append((node, intersection.distance))
    # [(node, dist), (node, dist)... ]
    pairs = sorted(pairs, key=lambda tup: tup[1])
    containers, _ = zip(*pairs)
    container = containers[0]
    return container


def next_hit(scene, ray):
    intersections = scene.intersections(ray.position, ray.direction)
    # Remove on surface intersections
    intersections = \
        [x for x in intersections if not close_to_zero(x.distance)]

    # Node which owns the surface
    if len(intersections) == 0:
        return None

    # The surface being hit
    hit = intersections[0]
    if len(intersections) == 1:
        hit_node = hit.hit
        return hit_node, (hit_node, None), hit.point, hit.distance
    
    container = find_container(intersections)
    hit = intersections[0]
    # Intersection point and distance from ray
    point = hit.point
    hit_node = hit.hit
    distance = distance_between(ray.position, point)
    if container == hit_node:
        adjacent = intersections[1].hit
    else:
        adjacent = hit_node
    return hit_node, (container, adjacent), point, distance


def trace(scene, ray, maxsteps=20):
    count = 0
    history = [ray]
    while True:
        count += 1
        if count > maxsteps:
            print("Max count reached.")
            break

        info = next_hit(scene, ray)
        print("Info: ", info)
        if info is None:
            print("[1] Exit.")
            break

        hit, (container, adjacent), point, full_distance = info
        print("interface: {}|{} (hit: {})".format(container, adjacent, hit))
        if hit is scene.root:
            print("[2] Exit.")
            break

        material = container.geometry.material
        absorbed, at_distance = material.is_absorbed(ray, full_distance)
        if absorbed:
            ray = ray.propagate(at_distance)
            component = material.component(ray.wavelength)
            if component.is_radiative(ray):
                ray = component.emit(ray)
                history.append(ray)
                print("Step", ray)
                continue
            else:
                history.append(ray)
                print("[3] Exit.")
                break
        else:
            ray = ray.propagate(full_distance)
            surface = hit.geometry.surface
            if surface.is_reflected(ray, hit.geometry, container, adjacent):
                ray = surface.reflect(ray, hit.geometry, container, adjacent)
                history.append(ray)
                print("REFLECT", ray)
                continue
            else:
                ray = surface.transmit(ray, hit.geometry, container, adjacent)
                history.append(ray)
                print("TRANSMIT", ray)
                continue
    return history


In [4]:
from pvtrace.scene.renderer import MeshcatRenderer
from pvtrace.scene.scene import Scene
from pvtrace.scene.node import Node
from pvtrace.light.ray import Ray
from pvtrace.geometry.sphere import Sphere
import numpy as np
np.random.seed(2)


air = Material(refractive_index=1.0)

plastic = Material(
    refractive_index=1.5,
    components=[
        Absorber(coefficient=1.0),
        Scatterer(coefficient=2.0, phase_function=lambda : henyey_greenstein(g=0.99)),
        Luminophore(
           coefficient=[lambda x: bandgap(x, 555, 2.0)],
           emission=[lambda x: gaussian(x, 1.0, 600, 50)],
           x=np.linspace(300, 800, 500),
           phase_function=isotropic
        )
    ]
)
world = Node(
    name="world",
    geometry=Sphere(
        radius=10,
        material=air,
        surface=Surface()
    )
)
ball = Node(
    name="ball",
    parent=world,
    geometry=Sphere(
        radius=1,
        material=plastic,
        surface=FresnelSurface()
    )
)
ray = Ray(position=(0, 0., 2), direction=(0, 0, -1), wavelength=540)
scene = Scene(root=world) 
vis = MeshcatRenderer()
vis.render(scene)
vis.vis.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [5]:
for _ in range(20):
    position = (
        ray.position[0] + np.random.uniform(-0.1, 0.1),
        ray.position[1],# + np.random.uniform(-0.1, 0.1),
        ray.position[2]
    )
    ray = replace(ray, position=position)
    path = trace(scene, ray)
    vis.add_ray_path(path)

INFO:__main__:Got alpha(540) = 0.0
INFO:__main__:Got alpha(540) = 0.0
INFO:__main__:Got alpha(540) = 5.0
INFO:__main__:Got alpha(540) = 5.0
INFO:__main__:Got alpha(572.8181570543431) = 3.0
INFO:__main__:Got alpha(572.8181570543431) = 3.0
INFO:__main__:Got alpha(572.8181570543431) = 3.0
INFO:__main__:Got alpha(540) = 0.0
INFO:__main__:Got alpha(540) = 5.0
INFO:__main__:Got alpha(540) = 0.0
INFO:__main__:Got alpha(540) = 5.0
INFO:__main__:Got alpha(540) = 5.0
INFO:__main__:Got alpha(664.5975427465386) = 3.0
INFO:__main__:Got alpha(664.5975427465386) = 3.0
INFO:__main__:Got alpha(664.5975427465386) = 3.0
INFO:__main__:Got alpha(540) = 0.0
INFO:__main__:Got alpha(540) = 5.0
INFO:__main__:Got alpha(556.4432682093073) = 3.0
INFO:__main__:Got alpha(556.4432682093073) = 3.0
INFO:__main__:Got alpha(556.4432682093073) = 3.0
INFO:__main__:Got alpha(556.4432682093073) = 3.0
INFO:__main__:Got alpha(540) = 0.0
INFO:__main__:Got alpha(540) = 5.0
INFO:__main__:Got alpha(581.6780955891563) = 3.0
INFO:_

Info:  (Node(ball), (Node(world), Node(ball)), (-0.01280101957159925, 0.0, 0.9999180635921763), 1.0000819364078237)
interface: Node(world)|Node(ball) (hit: Node(ball))
[]
Incident ray (0, 0, -1)
Reflection ((0, 0, -1), (-0.012801019571599253, 0.0, 0.9999180635921764))
Reflected ray [-0.02559994  0.          0.99967227]
REFLECT Ray(position=(-0.01, 0.00, 1.00), direction=(-0.03, 0.00, 1.00), wavelength=540.00, is_alive=True)
Info:  (Node(world), (Node(world), None), (-0.24320238002732525, 0.0, 9.997042192686248), 9.000073743099364)
interface: Node(world)|None (hit: Node(world))
[2] Exit.
Info:  (Node(ball), (Node(world), Node(ball)), (-0.002868523995857422, 0.0, 0.9999958857765789), 1.000004114223421)
interface: Node(world)|Node(ball) (hit: Node(ball))
[]
Refraction ((0, 0, -1), array([ 0.00286852, -0.        , -0.99999589]), 1.0, 1.5)
TRANSMIT Ray(position=(-0.00, 0.00, 1.00), direction=(0.00, 0.00, -1.00), wavelength=540.00, is_alive=True)
Info:  (Node(ball), (Node(ball), Node(world))

INFO:__main__:Got alpha(540) = 0.0
INFO:__main__:Got alpha(540) = 5.0
INFO:__main__:Got alpha(540) = 0.0
INFO:__main__:Got alpha(540) = 5.0
INFO:__main__:Got alpha(540) = 5.0
INFO:__main__:Got alpha(540) = 0.0
INFO:__main__:Got alpha(540) = 5.0
INFO:__main__:Got alpha(630.7701253384074) = 3.0
INFO:__main__:Got alpha(540) = 0.0
INFO:__main__:Got alpha(540) = 5.0
INFO:__main__:Got alpha(540) = 5.0
INFO:__main__:Got alpha(540) = 0.0
INFO:__main__:Got alpha(540) = 5.0
INFO:__main__:Got alpha(540) = 5.0
INFO:__main__:Got alpha(540) = 0.0
INFO:__main__:Got alpha(540) = 5.0
INFO:__main__:Got alpha(540) = 5.0
INFO:__main__:Got alpha(661.3746846337319) = 3.0
INFO:__main__:Got alpha(661.3746846337319) = 3.0
INFO:__main__:Got alpha(540) = 0.0
INFO:__main__:Got alpha(540) = 0.0
INFO:__main__:Got alpha(540) = 5.0
INFO:__main__:Got alpha(651.9818944292822) = 3.0
INFO:__main__:Got alpha(651.9818944292822) = 3.0
INFO:__main__:Got alpha(651.9818944292822) = 3.0
INFO:__main__:Got alpha(651.9818944292822

Info:  (Node(ball), (Node(world), Node(ball)), (-0.22852035686149424, 0.0, 0.9735391345497599), 1.02646086545024)
interface: Node(world)|Node(ball) (hit: Node(ball))
[]
Refraction ((0, 0, -1), array([ 0.22852036, -0.        , -0.97353913]), 1.0, 1.5)
TRANSMIT Ray(position=(-0.23, 0.00, 0.97), direction=(0.08, 0.00, -1.00), wavelength=540.00, is_alive=True)
Info:  (Node(ball), (Node(ball), Node(world)), (-0.07525615965846336, 0.0, -0.9971642344335561), 1.9766541636478754)
interface: Node(ball)|Node(world) (hit: Node(ball))
[1.0, 2.0, 2.0]
[3] Exit.
Info:  (Node(ball), (Node(world), Node(ball)), (-0.3094627464093335, 0.0, 0.9509115671737263), 1.0490884328262737)
interface: Node(world)|Node(ball) (hit: Node(ball))
[]
Refraction ((0, 0, -1), array([ 0.30946275, -0.        , -0.95091157]), 1.0, 1.5)
TRANSMIT Ray(position=(-0.31, 0.00, 0.95), direction=(0.11, 0.00, -0.99), wavelength=540.00, is_alive=True)
Info:  (Node(ball), (Node(ball), Node(world)), (-0.10080208366064838, 0.0, -0.99490649

INFO:__main__:Got alpha(540) = 5.0
INFO:__main__:Got alpha(598.7491880459256) = 3.0
INFO:__main__:Got alpha(598.7491880459256) = 3.0
INFO:__main__:Got alpha(598.7491880459256) = 3.0


 Ray(position=(-0.26, 0.00, 0.65), direction=(0.01, -0.00, 1.00), wavelength=540.00, is_alive=True)
Info:  (Node(ball), (Node(ball), Node(world)), (-0.2590026075404646, -0.0003986096834150044, 0.9658765399353895), 0.31252812223245546)
interface: Node(ball)|Node(world) (hit: Node(ball))
[1.0, 2.0, 2.0]
Step Ray(position=(-0.26, -0.00, 0.71), direction=(0.54, 0.50, 0.68), wavelength=598.75, is_alive=True)
Info:  (Node(ball), (Node(ball), Node(world)), (-0.049189334630012876, 0.1995436067982129, 0.9786535435712768), 0.39650234733848555)
interface: Node(ball)|Node(world) (hit: Node(ball))
[1.0, 2.0, 0.0]
Step Ray(position=(-0.24, 0.02, 0.74), direction=(0.05, 0.03, 1.00), wavelength=598.75, is_alive=True)
Info:  (Node(ball), (Node(ball), Node(world)), (-0.22635991404186148, 0.02861838299802759, 0.9736232215130961), 0.23370278483554477)
interface: Node(ball)|Node(world) (hit: Node(ball))
[1.0, 2.0, 0.0]
Step Ray(position=(-0.23, 0.02, 0.81), direction=(-0.02, -0.11, 0.99), wavelength=598.75